In [1]:
from utils.configuration import get_config_from_json
from utils.training_utilities import set_GPU
from utils.plotting_traces import plot_traces
from seq2point.seq2point import SEQ2POINT
import builtins
from pprint import pprint

builtins.GENERAL_CONFIG = get_config_from_json(description="General Settings", config_file="configs/general_config.json")
builtins.MODEL_CONFIG = get_config_from_json(description="Model Parameters", config_file="configs/model_config.json")
builtins.TRAINING_CONFIG = get_config_from_json(description="Training Configuration", config_file="configs/training_config.json")
builtins.PLOT_CONFIG = get_config_from_json(description="Plot Settings", config_file="configs/plot_config.json")

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data.dataset import TensorDataset
import torch
from refit_loader.data_loader import REFIT_Loader

In [3]:
refit = REFIT_Loader()

Followings are the general configuration of your experiment..
{'DATA_FOLDER': 'data/refit/', 'DATA_TYPE': '.csv', 'README_FILE': 'refit_loader/REFIT_Readme.txt', 'REFIT_HOUSES': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]}

Loading specified buildings: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]
Parsing the readme file specified: refit_loader/REFIT_Readme.txt


In [4]:
kettle = refit.get_appliance_data(appliance="Kettle", houses=[2])

Loading data for appliance KETTLE ...
Fetching KETTLE data for House 2


In [17]:
kettle.resample(sampling_period='8s', fill_value=0.0, window_limit=2.0)

Resampling for house number:  2


In [18]:
kettle.data[2].isnull().values.any()

False

In [19]:
kettle.data[2].isnull().sum().sum()

0

In [20]:
import numpy as np
# len(np.array(kettle.data[2]['aggregate'][:100]))

In [21]:
from data_generator import DataGenerator 

In [22]:
train_x= np.array(kettle.data[2]['aggregate'][0:10000])
train_y= np.array(kettle.data[2]['kettle'][0:10000] )
validate_x= np.array(kettle.data[2]['aggregate'][0:10000])
validate_y= np.array(kettle.data[2]['kettle'][0:10000])

In [23]:

def main(train_x, train_y, validate_x, validate_y):
    
    print(f"Followings are the {GENERAL_CONFIG['DESCRIPTION']} of your project..")
    pprint(GENERAL_CONFIG)
    
    ###### random seed selection in progress

    random_seed = 10


        
    network = SEQ2POINT().to(set_GPU())

#     train_mains,valid_mains,train_appliance,valid_appliance = train_test_split(mains, appliance, test_size=.2, random_state = random_seed)


#     train_dataset = TensorDataset(torch.from_numpy(train_mains).float(), torch.from_numpy(train_appliance).float())

#     validation_dataset = TensorDataset(torch.from_numpy(valid_mains).float(), torch.from_numpy(valid_appliance).float())

    generator = DataGenerator( train_x, train_y, validate_x, validate_y,
                              batch_size=1024,
                              seq_len=599,
                              width=1)

#     train_dataloader = torch.utils.data.DataLoader(dataset=generator, 
#                                   batch_size=20, # how many samples per batch
#                                   num_workers=1, # how many subprocesses to use for data loading (higher = more)
#                                   shuffle=True) # shuffle the data

#     print(train_dataloader)
#     for i, s in generator:
#         print(i)
#         print(s)

#     validation_dataloader = torch.utils.data.DataLoader(dataset=validation_dataset, 
#                                  batch_size=20, 
#                                  num_workers=1, 
#                                  shuffle=True) # don't need to shuffle testing data
    
    train_loss, validation_loss = network.run(generator.generate(), generator.generate())

    plot_traces(traces = [train_loss, validation_loss], labels=['training', 'validation'], axis_labels=['Epochs', 'Loss'], title='Training Loss vs Validation Loss per Epoch')


In [ ]:
main(train_x, train_y, validate_x, validate_y)

Followings are the General Settings of your project..
{'DATA_PATH': 'data/refit/',
 'DESCRIPTION': 'General Settings',
 'LOAD_MODEL': '',
 'PRE_TRAINED_MODEL_FLAG': False,
 'SAVE_PATH': 'models/'}

Initializing SEQ2POINT model archiecture

Followings are the Model Parameters of your network architecture..
{'CONV_KERNEL': [10, 8, 6, 5, 5],
 'CONV_LAYERS': 5,
 'CONV_PADDING': 0,
 'CONV_STRIDE': 1,
 'DESCRIPTION': 'Model Parameters',
 'INPUT_CHANNELS': [1, 30, 30, 40, 50],
 'LEFT_PAD': [4, 3, 2, 2, 2],
 'LINEAR_INPUT': [29950, 1024],
 'LINEAR_LAYERS': 2,
 'LINEAR_OUTPUT': [1024, 1],
 'OUTPUT_CHANNELS': [30, 30, 40, 50, 50],
 'POOL_KERNEL': [],
 'POOL_STRIDE': [],
 'RIGHT_PAD': [5, 4, 3, 2, 2],
 'SEQUENCE_LENGTH': 599}

SEQ2POINT model archiecture has been initialized

Followings are the Training Configuration of your experiment..
{'DESCRIPTION': 'Training Configuration',
 'EARLY_STOPPING_THRESHOLD': 3,
 'LEARNING_RATE': 0.001,
 'LOSS': 'nn.MSELoss',
 'LOSS_REDUCTION': 'mean',
 'NUM_EPOCHS